# DATA 608 - Homework 4
### LOGAN THOMSON

## Enterococcus Levels in the Hudson River
Using data from the organization Riverkeeper ( http://www.riverkeeper.org/).

**Background**: Enterococcus is a fecal indicating bacteria that lives in the intestines of humans and other warm-blooded animals. Enterococcus (“ Entero”) counts are useful as a water quality indicator due to their abundance in human sewage, correlation with many human pathogens and low abundance in sewage free environments. The United States Environmental Protection Agency (EPA) reports Entero counts as colonies (or cells) per 100 ml of water.

Riverkeeper has based its assessment of acceptable water quality on the 2012 Federal Recreational Water Quality Criteria from the US EPA. Unacceptable water is based on an illness rate of 32 per 1000 swimmers.

The federal standard for unacceptable water quality is a single sample value of greater than 110 Enterococcus/100 mL, or five or more samples with a geometric mean (a weighted average) greater than 30 Enterococcus/100 mL.

## LOAD PACKAGES  

For this assignment, Python and one of its imaging libraries will be utilized. In this case, I have chosen to utilize `Bokeh`. To pull the data from the GitHub repository, `urllib2` is used, along with `pandas` for data wrangling.  

In [75]:
import pandas as pd
# import numpy as np
import urllib2

from bokeh.charts import Bar, Scatter, Histogram, show # output_file,
from bokeh.layouts import row, gridplot, column
from bokeh.io import output_notebook
from bokeh.plotting import figure

import statsmodels.formula.api as smf

Data located at: https://github.com/charleyferrari/CUNY_DATA608/tree/master/lecture4/Data

## ACCESS RIVERKEEPER DATA

In [2]:
url = 'https://raw.githubusercontent.com/charleyferrari/CUNY_DATA608/master/lecture4/Data/riverkeeper_data_2013.csv'
response = urllib2.urlopen(url)

In [3]:
river = pd.read_csv(response, header=0)
river.head()

,Site,Date,EnteroCount,FourDayRainTotal,SampleCount
0,Hudson above Mohawk River,10/16/2011,1733,1.5,35
1,Hudson above Mohawk River,10/21/2013,4,0.2,35
2,Hudson above Mohawk River,9/21/2013,20,0.0,35
3,Hudson above Mohawk River,8/19/2013,6,0.0,35
4,Hudson above Mohawk River,7/21/2013,31,0.0,35


In [4]:
river.dtypes

Site                 object
Date                 object
EnteroCount          object
FourDayRainTotal    float64
SampleCount           int64
dtype: object

The Riverkeeper data is fairly straightforward, consisting of 5 columns for the testing site name, the date of the sample is taken, the count of entero in the water sample, the previous four days rain total (**more than 1/4 inch is considered a “wet weather sample.”**), and the total number of samples taken from that site in the dataset.  

We see above that the data type for the `Date` and `EnteroCount` are in pandas object format, and will need to be converted in order to visualize and analyze appropriately.  

#### Convert `Date` column to date format

In [91]:
river['Date'] = pd.to_datetime(river.Date)

#### Clean up `EnteroCount` values

The `EnteroCount` column contains values, which reflect the amount of Enterococcus in the water sample. Entero is a fecal indicating bacterium that lives in the intestines of humans and warm-blooded animals.  

The values in this column contain the amounts, some are preceded by ">" or "<", which indicate a value greater than the one specified (i.e. >2420) and reflects the upper (>) or lower (<) limit of Riverkeeper's scoring ability for that sample.

Because these indicators might be useful later on, we'll save them to a new column simply called `AboveBelow`. The Python script below will add this symbol in the new column if it exists in the `EnteroCount` column.  Next, the same symbols will be stripped from the values in the `EnteroCount` column.  

In [6]:
# Create a column with > and < from EnteroCount column

river['AboveBelow'] = ['>' if '>' in x else
                    '<' if '<' in x else
                    '' for x in river['EnteroCount']]

river['EnteroCount'] = river['EnteroCount'].map(lambda x: x.lstrip('><'))  # remove "<" and ">" from EnteroCount

After removing the symbols the `EnteroCount` values are converted to numeric values:

In [7]:
river['EnteroCount'] = pd.to_numeric(river['EnteroCount'], errors='coerce')

Lastly, the columns are reordered, and a sample of the new data is below:  

In [8]:
river = river[['Site', 'Date', 'AboveBelow', 'EnteroCount', 'FourDayRainTotal', 'SampleCount']]
river.head()

,Site,Date,AboveBelow,EnteroCount,FourDayRainTotal,SampleCount
0,Hudson above Mohawk River,2011-10-16,,1733,1.5,35
1,Hudson above Mohawk River,2013-10-21,,4,0.2,35
2,Hudson above Mohawk River,2013-09-21,,20,0.0,35
3,Hudson above Mohawk River,2013-08-19,,6,0.0,35
4,Hudson above Mohawk River,2013-07-21,,31,0.0,35


After converting and doing some slight clean-up work, the data types of each column is confirmed below.  

In [9]:
river.dtypes

Site                        object
Date                datetime64[ns]
AboveBelow                  object
EnteroCount                  int64
FourDayRainTotal           float64
SampleCount                  int64
dtype: object

In [10]:
output_notebook()

Loading BokehJS ...

## Create Lists & Graphs of the Best and Worst places to Swim in the Dataset.

Top 10 best and worst places to swim, based on their average entero levels are created below by collapsing the data using `pandas pivot.table` function, then sorting for average entero level.  

### Worst Places to Swim

In [11]:
#river.pivot_table('EnteroCount', 'Site', aggfunc='sum')  # summed EnteroCount
entero_avg = river.pivot_table('EnteroCount', 'Site')  # average EnteroCounts
entero_avg = entero_avg.reset_index()
worst = entero_avg.sort_values(by='EnteroCount', ascending=False)[:10]  # Top 10 worst places to swim
worst = pd.DataFrame(worst)
worst

,Site,EnteroCount
29,Gowanus Canal,4206.837838
48,Newtown Creek- Metropolitan Ave. Bridge,2953.684211
66,Tarrytown Marina,2205.666667
63,Saw Mill River,1455.760000
70,Upper Sparkill Creek,1296.072727
47,Newtown Creek- Dutch Kills,1205.087719
39,Kingsland Pt. Park- Pocantico River,907.857143
53,Orangetown STP Outfall,854.192982
45,Mohawk River at Waterford,621.057143
57,Piermont Pier,482.165775


In [12]:
worst_plot = Bar(worst, 'Site', values='EnteroCount', title="Highest Average EnteroCount", 
                 color='lightgreen', plot_height=700, plot_width=900, legend=False)

show(row(worst_plot))

According to Riverkeeper, the sample level of entero which would require a beach advisory is any count over 60 for a single sample.  Looking at the chart and table above, almost every site in the 10 worst places to swim has an average entero count almost 10 times the warning limit for a beach advisory. It would be highly advisable to avoid swimming in any of the above locations, regardless of past rain fall amounts.  

### Best Places to Swim

In [13]:
best = entero_avg.sort_values(by='EnteroCount', ascending=True)[:10]  # Top 10 best places to swim
best = pd.DataFrame(best)
best

,Site,EnteroCount
59,Poughkeepsie Drinking Water Intake,8.342105
17,Croton Point Beach,15.458333
64,Stony Point mid-channel,17.340909
42,Little Stony Point,17.526316
60,Poughkeepsie Launch Ramp,17.675676
32,Haverstraw Bay mid-channel,18.708333
65,TZ Bridge mid-channel,21.438596
14,Cold Spring Harbor,22.542857
74,Yonkers mid-channel,25.019231
37,Irvington Beach,28.805556


In [14]:
best_plot = Bar(best, 'Site', values='EnteroCount', title="Lowest EnteroCount Average", 
                color='mediumaquamarine', plot_height=700, plot_width=900,legend=False)

show(row(best_plot))

The top 10 best places to swim all have average entero counts below 30, which is in range of Riverkeeper's accaptable range for a geometric mean (0-30).  Thankfully, the Poughkeepsie Drinking Water Intake site has an average entero count under 10.  

## Testing Frequency

### Which sites have been tested most regularly?
- The testing of water quality can be sporadic.

### 20 Sites tested most regularly

In [15]:
site_count = river.pivot_table('SampleCount', 'Site')  # average EnteroCounts
site_count = site_count.reset_index()
most_test = site_count.sort_values(by='SampleCount', ascending=False)[:20]  # Top 20 most

The `SampleCount` column already contains the total amount of times a site has been sampled from, but if we want to see the distribution of sampling counts, the data will need to be collapsed so that the count does not reflect the sum of each row of data for each testing site.  

The histogram below shows that most of the sites have been tested between 35-50 times. Two of the sites have had much higher testing frequency.

In [16]:
d = Histogram(site_count, values='SampleCount', bins=100, title="Distribution of Sample Counts", plot_width=900)

show(d)

### Most Frequently Tested

The 20 sites which have been tested the most regularly are below.

In [17]:
most_test

,Site,SampleCount
57,Piermont Pier,187
70,Upper Sparkill Creek,165
0,125th St. Pier,66
52,Nyack Launch Ramp,61
47,Newtown Creek- Dutch Kills,57
65,TZ Bridge mid-channel,57
53,Orangetown STP Outfall,57
48,Newtown Creek- Metropolitan Ave. Bridge,57
74,Yonkers mid-channel,52
73,Yonkers STP Outfall,51


### Least Tested

Reversing the sorting of the site_count DataFrame, the sites with the least amount of testing are below. The range of the testing dates ranges from September 2006 to October 2013 (2500+ days), so the testing is averaging once every two months.

As we'll see in the plots below, much of the testing is clustered into certain time frames for most of the sites.  

In [18]:
max(river['Date']) - min(river['Date'])

Timedelta('2589 days 00:00:00')

In [19]:
least_test = site_count.sort_values(by='SampleCount', ascending=True)[:20]  # 20 least tested sites

least_test

,Site,SampleCount
66,Tarrytown Marina,27
14,Cold Spring Harbor,35
35,Hudson above Mohawk River,35
38,Island Creek/Normans Kill,35
44,Marlboro Landing,35
16,Coxsackie Waterfront Park,35
45,Mohawk River at Waterford,35
8,Castleton,35
71,Wappingers Creek,35
4,Athens,35


### Which ones have long gaps between tests? 
- Pick out 5-10 sites and visually compare how regularly their water quality is tested.  

Since the number of total tests would indicate how often the site is tested, the river data is subsetted with the sites with the lowest sample counts, and combined with those with the highest testing counts.  

In [20]:
low_high_test = river[(river['SampleCount'] < 37) | (river['SampleCount'] >= 60) ]

In [21]:
p = Scatter(low_high_test, x='Date', y='Site', color='SampleCount', title="Site Testing Dates", 
            plot_width = 1000, plot_height=750, legend=False, tooltips=[('EnteroCount', '@EnteroCount'),
                                                                       ('4-Day Rain Total', '@FourDayRainTotal')])

show(p)

Examining the plot above, the sites with the lower sample counts are only tested in warmer weather (May-Sept.), about once per month. One site in particular (Tarrytown Marina) which also has the lowest testing frequency, went almost a full year without a sample being taken.  

This contrasts greatly with the sites with a sample frquency almost double that of the lowest testing frequency sites, where we can see that the sampling can take place multiple times in a month. If we wanted to know the effect of other factors (weather, other environmental conditions), it would appear tha much higher rates of frequency of sampling are required.  

### Is there a relationship between the amount of rain and water quality? 
Show this relationship graphically. If you can, estimate the effect of rain on quality at different sites and create a visualization to compare them.

In [82]:
p = Scatter(river, x='EnteroCount', y='FourDayRainTotal', title="Rain Totals vs. EnteroCount", 
            plot_width = 950, legend=False)

show(p) 

Examining the scatter plot showing the relationship of entero amounts to the previous rain fall amounts, we can see that many of the larger entero levels also follow rain fall amounts above .25". 

Of course, some of these sites have already high amounts of entero, and many of the measurements have been taken when the rainfall amounts are not very significant. With these varying amounts, and all of the testing sites tested at different times, it is hard to draw a conclusion.  In addition, the data for entero contained the values with the ">" and "<" symbols, which indicated an amount that was much higher or lower than the normal for that site.  

Performing a simple regression analysis using only the `EnteroCount` and `FourDayRainTotal` variables yields a very low correlation between the two.  

### Linear Regression

In [113]:
formula = 'EnteroCount ~ FourDayRainTotal'
results = smf.ols(formula, data=river).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            EnteroCount   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     72.73
Date:                Sat, 25 Mar 2017   Prob (F-statistic):           2.20e-17
Time:                        13:34:08   Log-Likelihood:                -30681.
No. Observations:                3397   AIC:                         6.137e+04
Df Residuals:                    3395   BIC:                         6.138e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept          219.4968     39.952      5.494      0.000       141.165   297.829
FourDayRainTotal   296.2157     34.733      8.528      0.000       228.117   364.315
==============================================================================
Omnibus:                     5326.395   Durbin-Watson:                   1.751
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1840481.660
Skew:                          10.179   Prob(JB):                         0.00
Kurtosis:                     115.199   Cond. No.                         1.75
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Using some of the points made above, let's subset the data to include only samples taken after wet weather, with entero counts above the acceptable limit for a single sample (60).  

Two different plots are made, one to separate out the test samples far above the sample range for the site, and the other to show those that are in the normal range.  

In [38]:
above_norm_wet = river[(river['AboveBelow'] == '>')]

p = Scatter(above_norm_wet, x='EnteroCount', y='FourDayRainTotal', 
            title="Rain Totals vs. EnteroCount", plot_width=450, legend=False)

Abvbel_blank_wet = river[(river['AboveBelow'] == '') & (river['FourDayRainTotal'] > .25) & (river['EnteroCount'] > 60)]

p2 = Scatter(Abvbel_blank_wet, x='EnteroCount', y='FourDayRainTotal', 
             title="Rain Totals vs. EnteroCount", plot_width=450, legend=False)

show(row(p, p2))

Examining the plots above, the one on the left contains the samples where the entero levels are much higher than the normal level, without filtering out any rainfall levels. Interestingly enough, the large majority of the points for these larger levels of entero are related to higher rainfall levels, many more than 1/2".  


Below I selected a few of the sites in the dataset to see if there was any pattern to the rainfall totals and the increased levels of entero in the sample.  

In [85]:
sawmill = river[(river['Site'] == 'Saw Mill River')]
sawmill = sawmill.sort_values(by='Date')

gowanus = river[(river['Site'] == 'Gowanus Canal') & ([river['FourDayRainTotal']] > .25)]
gowanus = gowanus.sort_values(by='Date')

piermont = river[(river['Site'] == 'Piermont Pier') & ([river['FourDayRainTotal']] > .25)]
piermont = piermont.sort_values(by='Date')
#best = entero_avg.sort_values(by='EnteroCount', ascending=True)[:10]  

### Rainfall Totals and Entero Level: Piermont Pier  

I could not get two line charts to be drawn on the same plot (with two different y-axes), so below are two scatter plots, both with the date on the x-axis, and then entero counts and rain fall totals (respectively) on the y-axis. 

I chose the Piermont Pier location, since it has the highest frequency of tests. To allow ease of comparing the points, tooltips for both plots were generated.  

In [90]:
tooltips=[
    ('Entero', '@EnteroCount'),
    ('4-Day Rain Total', '@FourDayRainTotal')
]

p_plot = Scatter(piermont, 'Date', 'EnteroCount', color="firebrick", 
                 plot_height=350, plot_width=800, title="Piedmont Pier, EnteroCount by Date", tooltips=tooltips)

p_plot2 = Scatter(piermont, 'Date', 'FourDayRainTotal', color="navy", 
                  plot_height=350, plot_width=800, title="Piedmont Pier, Rain Totals by Date",tooltips=tooltips)

show(column(p_plot, p_plot2))

After looking at the plots, there are some spikes in the entero levels immediately following rain, but some higher levels are preceded by almost no rain. Some increased levels seems to occur from consecutive days of rain fall, or more rain in a shorter period. In fact, some of the lower entero counts were preceded by higher rainfall totals.  

Oddly enough for this location, some of the higher levels seem to occur in the colder months (Nov. - March). The reason may be that the lack of new rain fall (or snow melt) to flush out natural water sources will result in higher entero levels.  

Other factors that would require further analysis (and more data) is the geographical location of these sites. Are they downstream from pollution sources (see: Gowanus and Newtown Creek)? Does the elevation or type of water source (river, creek, etc.) matter as well?